In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

In [2]:
np.random.seed(42)
df = pd.read_csv("/kaggle/input/quora-insincere-questions-classification/train.csv")

In [4]:
df.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [5]:
df = df.drop("qid", axis=1)
df.head(10)

,question_text,target
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0
5,"Is Gaza slowly becoming Auschwitz, Dachau or T...",0
6,Why does Quora automatically ban conservative ...,0
7,Is it crazy if I wash or wipe my groceries off...,0
8,"Is there such a thing as dressing moderately, ...",0
9,Is it just me or have you ever been in this ph...,0


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:


x = df["question_text"]
y = df["target"]

vectorizer = TfidVectorizer(
    max_features = 10000,
    ngram_range = (1,2),
    stopwords = "english"
)

x_tfidf = vectorizer.fit_transform(x)

df_balanced = pd.DataFrame({'text': x, 'target': y})

class_0 = df_balanced[df_balanced['target'] == 0]
class_1 = df_balanced[df_balanced['target'] == 1]

# Find minority class size
min_size = min(len(class_0), len(class_1))

# Downsample majority class
class_0_downsampled = class_0.sample(min_size, random_state=42)
class_1_downsampled = class_1.sample(min_size, random_state=42)

# Combine and shuffle
df_resampled = pd.concat([class_0_downsampled, class_1_downsampled]).sample(frac=1, random_state=42).reset_index(drop=True)

# Final feature and target after resampling
x_resampled = vectorizer.transform(df_resampled['text'])
y_resampled = df_resampled['target']

NameError: name 'TfidVectorizer' is not defined